---------------------------------------

# $SPA$tial $G$rap$H$s: n$ET$works, $T$opology, & $I$nference

## Tutorial for `pysal.spaghetti`: Working with point patterns: empirical observations
#### James D. Gaboardi [<jgaboardi@fsu.edu>]

1. Instantiating a `pysal.spaghetti.Network`
2. Allocating observations to a network
    * snapping
3. Visualizing original and snapped locations
    * visualization with `geopandas` and `matplotlib`

In [ ]:
import os
last_modified = None
if os.name == "posix":
    last_modified = !stat -f\
                    "# This notebook was last updated: %Sm"\
                     Spaghetti_Pointpatterns_Empirical.ipynb
elif os.name == "nt":
    last_modified = !for %a in (Spaghetti_Pointpatterns_Empirical.ipynb)\
                    do echo # This notebook was last updated: %~ta
    
if last_modified:
    get_ipython().set_next_input(last_modified[-1])

In [ ]:
# This notebook was last updated: Nov 24 16:12:11 2018

-----------------

In [ ]:
import spaghetti as spgh
from libpysal import examples
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from shapely.geometry import Point, LineString

%matplotlib inline

__author__ = "James Gaboardi <jgaboardi@gmail.com>"

# 1. Instantiating a `pysal.spaghetti.Network`
### Instantiate the network from `.shp` file

In [ ]:
ntw = spgh.Network(in_data=examples.get_path('streets.shp'))

# 2. Allocating observations to a network
### Snap point patterns to the network

In [ ]:
# Crimes with attributes
ntw.snapobservations(examples.get_path('crimes.shp'), 'crimes', attribute=True)
# Schools without attributes
ntw.snapobservations(examples.get_path('schools.shp'), 'schools', attribute=False)

### A network is composed of a single topological representation of a road and $n$ point patterns which are snapped to the network.

In [ ]:
ntw.pointpatterns

In [ ]:
dir(ntw.pointpatterns['crimes'])

### Attributes for every point pattern

1. `dist_to_node` dict keyed by pointID with the value being a dict in the form 
        {node: distance to node, node: distance to node}
2. `obs_to_edge` dict keyed by edgeID (tuple) with the value being a dict in the form 
        {pointID:(x-coord, y-coord), pointID:(x-coord, y-coord), ... }
3. `obs_to_node` dict keyed by nodeID with the value being a list of pointIDs
        {nodeID:[pointID]}
4. `points` geojson like representation of the point pattern.  Includes properties if read with attributes=True.
        {pointID: {'coordinates': (x-coord, y-coord), 'properties': [[attributes, attributes,...]]}
5. `snapped_coordinates` dict keyed by pointID with the value being (x-coord, y-coord)
        {pointID: (x-coord, y-coord)}

# 3. Visualizing original and snapped locations

## School locations

In [ ]:
school_points = ntw.pointpatterns['schools'].points
print(type(school_points))
print(school_points[0])

### **  The original coordinates of the schools need to be extracted from `school_points`

In [ ]:
schools = range(len(school_points))
school_coords = {school:school_points[school]["coordinates"] for school in schools}
print(type(school_coords))
print(school_coords[0])

## Snapped school locations

In [ ]:
snapped_school_coords = ntw.pointpatterns['schools'].snapped_coordinates
print(type(snapped_school_coords))
print(snapped_school_coords[0])

## Crime locations

In [ ]:
crime_points = ntw.pointpatterns['crimes'].points
print(type(crime_points))
print(crime_points[0])

### **  The original coordinates of the crimes need to be extracted from `crime_points`

In [ ]:
crimes = range(len(crime_points))
crime_coords = {crime:crime_points[crime]["coordinates"] for crime in crimes}
print(type(crime_coords))
print(crime_coords[0])

## Snapped crime locations

In [ ]:
snapped_crime_coords = ntw.pointpatterns['crimes'].snapped_coordinates
print(type(snapped_crime_coords))
print(snapped_crime_coords[0])

## Create `geopandas.GeoDataFrame` objects of the nodes and edges

In [ ]:
# network nodes and edges
nodes_df, edges_df = spgh.element_as_gdf(ntw, nodes=ntw.node_coords,
                                         edges=ntw.edges)

In [ ]:
# schools and snapped schools
schools_df = spgh.element_as_gdf(ntw, pp_name='schools', snapped=False)
snapped_schools_df = spgh.element_as_gdf(ntw, pp_name='schools', snapped=True)

In [ ]:
# crimes and snapped crimes
crimes_df = spgh.element_as_gdf(ntw, pp_name='crimes', snapped=False)
snapped_crimes_df = spgh.element_as_gdf(ntw, pp_name='crimes', snapped=True)

## Plotting `geopandas.GeoDataFrame` objects

In [ ]:
# legend patches
strs = mlines.Line2D([], [], color='k', label='Network Segments', alpha=.5)
ndes = mlines.Line2D([], [], color='k', linewidth=0, markersize=2.5,
                     marker='o', label='Network Nodes', alpha=1)
schl = mlines.Line2D([], [], color='k', linewidth=0, markersize=25,
                     marker='X', label='School Locations', alpha=1)
snp_schl = mlines.Line2D([], [], color='k', linewidth=0, markersize=12,
                         marker='o', label='Snapped Schools', alpha=1)
crme = mlines.Line2D([], [], color='r', linewidth=0, markersize=7,
                     marker='x', label='Crime Locations', alpha=.75)
snp_crme = mlines.Line2D([], [], color='r', linewidth=0, markersize=3,
                         marker='o', label='Snapped Crimes', alpha=.75)

patches = [strs, ndes, schl, snp_schl, crme, snp_crme]

In [ ]:
# plot figure
base = edges_df.plot(color='k', alpha=.25, figsize=(12,12), zorder=0)
nodes_df.plot(ax=base, color='k', markersize=5, alpha=1)

crimes_df.plot(ax=base, color='r', marker='x',
               markersize=50, alpha=.5, zorder=1)
snapped_crimes_df.plot(ax=base, color='r',
                       markersize=20, alpha=.5, zorder=1)

schools_df.plot(ax=base, cmap='tab20', column='id', marker='X',
                markersize=500, alpha=.5, zorder=2)
snapped_schools_df.plot(ax=base,cmap='tab20', column='id',
                        markersize=200, alpha=.5, zorder=2)

# add legend
plt.legend(handles=patches, fancybox=True, framealpha=0.8,
           scatterpoints=1, fontsize="xx-large", bbox_to_anchor=(1.04, .6))

-----------